<a name='introduction'></a>
# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нужно построить модель для определения стоимости. 

Заказчику важны:
- качество предсказания;
- скорость предсказания;
- время обучения.

<br>**План работы:**
1. [Подготовка данных](#data_preprocessing)
    - [Импорт и первичный анализ](#import)
    - [Предобработка количественных параметров](#numeric_cols)
    - [Удаление дубликатов и переустановка индексов](#duplicates)
2. [Обучение моделей](#model_learning)
    - [Разбивка на выборки и преобразование столбцов](#samples_split)
    - [Модели машинного обучения](#ml_models)
        - [LinearRegression](#logreg)
        - [DecisionTreeRegressor](#tree)
        - [LightGBM](#lgbm)
3. [Анализ моделей](#analysis)
4. [Общий вывод](#conclusion)

<a name='data_preprocessing'></a>
## 1. Подготовка данных

<a name='import'></a>
### 1. 1. Импорт и первичный анализ 

In [1]:
import numpy as np
import pandas as pd

from lightgbm import LGBMRegressor

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

In [2]:
# Импорт данных

df_autos = pd.read_csv("car_price_prediction.csv")

In [4]:
df_autos.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


In [5]:
# Проверим тип данных параметров

df_autos.dtypes

ID                    int64
Price                 int64
Levy                 object
Manufacturer         object
Model                object
Prod. year            int64
Category             object
Leather interior     object
Fuel type            object
Engine volume        object
Mileage              object
Cylinders           float64
Gear box type        object
Drive wheels         object
Doors                object
Wheel                object
Color                object
Airbags               int64
dtype: object

In [6]:
# Отсутствующие данные по параметрам

df_autos.isna().sum()

ID                  0
Price               0
Levy                0
Manufacturer        0
Model               0
Prod. year          0
Category            0
Leather interior    0
Fuel type           0
Engine volume       0
Mileage             0
Cylinders           0
Gear box type       0
Drive wheels        0
Doors               0
Wheel               0
Color               0
Airbags             0
dtype: int64

In [7]:
# Рассмотрим значения по столбцам с отсутствующими значениями

cols = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Repaired']

for col in cols:
    print(f"Unique values by '{col}':", df_autos[col].unique(), '\n')

KeyError: 'VehicleType'

In [7]:
# Сразу заменим значение в параметре 'Model' с 'rangerover' на 'range_rover'

df_autos.loc[df_autos['Model'] == 'rangerover', 'Model'] = 'range_rover'

Значения отсутствуют в категориальных переменных. Поскольку заполнение их самым частым встречающимся значением может исказить результаты работы конечной модели, то лучше было бы заполнить отсутствующие значения чем-то опреденным, что, в то же самое время, говорило о том, что значение неизвестно. В связи с этим можно взять за такое значение "unknown".

In [8]:
df_autos = df_autos.fillna('unknown')
df_autos.isna().sum()

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
Repaired             0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64

In [9]:
# Удалим лишние параметры (например, которые сильно коррелируют с другими или которые не несут важной информации)

df_autos = df_autos.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures', 'PostalCode'], axis=1)

Далее рассмотрим каждый из количественных параметров на наличие анамолий

<a name='numeric_cols'></a>
### 1. 2. Предобработка количественных параметров

#### "Price"

In [10]:
df_autos['Price'].sort_values()

205926        0
204097        0
204099        0
204104        0
62604         0
          ...  
214341    20000
66054     20000
197617    20000
143161    20000
113554    20000
Name: Price, Length: 354369, dtype: int64

Очевидно, что автомобили не будут продаваться бесплатно. Ввиду этого, ограничим минимальное значение цены, путем расчета 5%-ого квантиля параметра и использования его в качестве нижнего предела

In [11]:
price_quantile_05 = np.quantile(df_autos['Price'], q=0.05)
df_autos = df_autos[df_autos['Price'] > price_quantile_05]

#### "RegistrationYear"

In [12]:
df_autos['RegistrationYear'].sort_values()

112768    1000
66198     1000
244092    1000
91869     1000
143621    1000
          ... 
29426     9999
28965     9999
227462    9999
17271     9999
128677    9999
Name: RegistrationYear, Length: 334921, dtype: int64

In [13]:
year_quantile_005 = np.quantile(df_autos['RegistrationYear'], q=0.005)
year_quantile_99 = np.quantile(df_autos['RegistrationYear'], q=0.99)
print("0.005 quantile:", year_quantile_005)
print("0.99 quantile:", year_quantile_99)

0.005 quantile: 1972.0
0.99 quantile: 2018.0


Очевидно, что год регистрации автомбиля не может быть как 1000, так и 9999, поэтому также ограничим данный параметр, но уже и нижним, и верхним пределом, однако в данном случае будем использовать уже 0.5%-ый квантиль, поскольку он составляет 1980 год, что вполне может соответствовать действительности. Верхним пределом сделаем значение "2023".

In [14]:
df_autos = df_autos[(df_autos['RegistrationYear'] >= year_quantile_005) & (df_autos['RegistrationYear'] <= 2023)]

#### "Power"

In [15]:
df_autos['Power'].sort_values()

0             0
260495        0
67720         0
260486        0
260471        0
          ...  
63986     17932
132485    19208
114106    19211
299180    19312
219584    20000
Name: Power, Length: 333361, dtype: int64

Ввиду того, что минимальные и максимальные значения явно аномальные, необходимо также ограничить допустимые значения мощности автомобилей нижним и верхним пределами

In [16]:
power_quantile_09 = np.quantile(df_autos['Power'], q=0.09)
power_quantile_10 = np.quantile(df_autos['Power'], q=0.1)
power_quantile_99 = np.quantile(df_autos['Power'], q=0.99)
print("0.09 quantile:", power_quantile_09)
print("0.10 quantile:", power_quantile_10)
print("0.99 quantile:", power_quantile_99)

0.09 quantile: 0.0
0.10 quantile: 34.0
0.99 quantile: 300.0


Поскольку даже 9%-квантиль демонстрирует нулевое значение, то принято решение в качестве нижнего предела взять 12%-ный квантиль, а в качестве верхнего - 99%-ный квантиль.

Заполним пропускими медианными значениями мощности, сгруппировав их по брендам автомобилей. Оставим все, что находится в рамках от 10%-ного квантиля (первоначально взятого) и до 99%-ного квантиля.

In [17]:
median_brands_power = df_autos.groupby('Brand')[['Power']].median()

# Замена нулевой мощности медианной по бренду
for i, row in df_autos[df_autos['Power'] < power_quantile_10].iterrows():
    df_autos.loc[i, 'Power'] = median_brands_power.loc[row['Brand'], 'Power']
    
df_autos = df_autos[(df_autos['Power'] >= power_quantile_10) & (df_autos['Power'] <= power_quantile_99)]

#### "Kilometer"

In [18]:
df_autos['Kilometer'].sort_values()

97209       5000
304846      5000
147023      5000
255825      5000
336506      5000
           ...  
141522    150000
141523    150000
141524    150000
141472    150000
354368    150000
Name: Kilometer, Length: 329698, dtype: int64

#### "RegistrationMonth"

In [19]:
print("Уникальные значения 'RegistrationMonth':\n", sorted(df_autos['RegistrationMonth'].unique()))

Уникальные значения 'RegistrationMonth':
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


Значения данного параметра лежат в диапазоне от 0 до 12 (всего 13 чисел), однако количество месяцев 12. Ввиду того, что количество объектов и по 0 и по 12 месяца совпадает, то для избежания искажения информации оставим все как есть.

<a name='duplicates'></a>
### 1.  3. Удаление дубликатов и переустановка индексов

In [20]:
# Оставим датасет без дубликатов

df_autos = df_autos[~df_autos.duplicated()]

In [21]:
# Переустановим индексы объектов

df_autos = df_autos.reset_index(drop=True)

In [22]:
# Окончательный датасет после отчистки и замены отсутствующих значений

df_autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304170 entries, 0 to 304169
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              304170 non-null  int64 
 1   VehicleType        304170 non-null  object
 2   RegistrationYear   304170 non-null  int64 
 3   Gearbox            304170 non-null  object
 4   Power              304170 non-null  int64 
 5   Model              304170 non-null  object
 6   Kilometer          304170 non-null  int64 
 7   RegistrationMonth  304170 non-null  int64 
 8   FuelType           304170 non-null  object
 9   Brand              304170 non-null  object
 10  Repaired           304170 non-null  object
dtypes: int64(5), object(6)
memory usage: 25.5+ MB


<a name='model_learning'></a>
## 2. Обучение моделей
[Вернуться во Введение](#introduction)

<a name='samples_split'></a>
### 2. 1. Разбивка на выборки и преобразование столбцов

In [23]:
X = df_autos.drop('Price', axis=1)
y = df_autos['Price']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, test_size=0.25)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, random_state=42, test_size=0.5)

In [24]:
categorical_cols = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']
numeric_cols = ['RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']

numeric_transformer = StandardScaler()
categorical_transformer = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

preprocessor = ColumnTransformer(transformers=[
    ('numeric_transformer', numeric_transformer, numeric_cols),
    ('categorical_transformer', categorical_transformer, categorical_cols)
])

<a name='ml_models'></a>
### 2. 2. Модели машинного обучения

<a name='logreg'></a>
#### 2. 2. 1. LinearRegression

Обучение и прогнозирование модели LinearRegression

In [25]:
%%time

# Обучение модели LinearRegression
linear = LinearRegression()
linear_pipe = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', linear)])
linear_pipe.fit(X_train, y_train)

CPU times: user 370 ms, sys: 40.1 ms, total: 410 ms
Wall time: 377 ms


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric_transformer',
                                                  StandardScaler(),
                                                  ['RegistrationYear', 'Power',
                                                   'Kilometer',
                                                   'RegistrationMonth']),
                                                 ('categorical_transformer',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['VehicleType', 'Gearbox',
                                                   'Model', 'FuelType', 'Brand',
                                                   'Repaired'])])),
                ('regressor', LinearRegression())])

In [26]:
%%time

# Предсказания модели LinearRegression
y_pred_linear = linear_pipe.predict(X_valid)

rmse_linear = mean_squared_error(y_valid, y_pred_linear) ** 0.5
print("LinearRegression RMSE: {:.3f}".format(rmse_linear))

LinearRegression RMSE: 2889.069
CPU times: user 41.3 ms, sys: 63.2 ms, total: 104 ms
Wall time: 125 ms


<a name='tree'></a>
#### 2. 2. 2. DecisionTreeRegressor

In [27]:
'''
tree = DecisionTreeRegressor(random_state=42)
params_tree = {
    'max_depth': range(2, 21, 2),
    'min_samples_split': range(2, 21, 2)
}

grid = GridSearchCV(tree, params_tree)
grid.fit(X_train, y_train)
print(grid.best_estimator_)
'''

"\ntree = DecisionTreeRegressor(random_state=42)\nparams_tree = {\n    'max_depth': range(2, 21, 2),\n    'min_samples_split': range(2, 21, 2)\n}\n\ngrid = GridSearchCV(tree, params_tree)\ngrid.fit(X_train, y_train)\nprint(grid.best_estimator_)\n"

DecisionTreeRegressor(max_depth=16, min_samples_split=20, random_state=42)

In [28]:
%%time

# Обучение модели DecisionTreeRegressor
tree = DecisionTreeRegressor(random_state=42, max_depth=16, min_samples_split=20)
tree_pipe = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', tree)])
tree_pipe.fit(X_train, y_train)

CPU times: user 1.02 s, sys: 28 ms, total: 1.05 s
Wall time: 1.05 s


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric_transformer',
                                                  StandardScaler(),
                                                  ['RegistrationYear', 'Power',
                                                   'Kilometer',
                                                   'RegistrationMonth']),
                                                 ('categorical_transformer',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['VehicleType', 'Gearbox',
                                                   'Model', 'FuelType', 'Brand',
                                                   'Repaired'])])),
                ('regressor',
                 DecisionTreeRegressor(max_depth=16, min_samples_split=20,
               

In [29]:
%%time

# Предсказания модели DecisionTreeRegressor
y_pred_tree = tree_pipe.predict(X_valid)
rmse_tree = mean_squared_error(y_valid, y_pred_tree) ** 0.5
print("DecisionTreeClassifier RMSE: {:.3f}".format(rmse_tree))

DecisionTreeClassifier RMSE: 1841.436
CPU times: user 59.3 ms, sys: 0 ns, total: 59.3 ms
Wall time: 63.8 ms


<a name='lgbm'></a>
#### 2. 2. 3. LightGBM

In [30]:
'''
lgb = LGBMRegressor(random_state=42)
params_lgb = {
    'num_leaves': range(31, 38, 2),
    'n_estimators': range(100, 201, 50),
}

grid = GridSearchCV(lgb, params_lgb)
grid.fit(X_train, y_train)
print(grid.best_estimator_)
'''

"\nlgb = LGBMRegressor(random_state=42)\nparams_lgb = {\n    'num_leaves': range(31, 38, 2),\n    'n_estimators': range(100, 201, 50),\n}\n\ngrid = GridSearchCV(lgb, params_lgb)\ngrid.fit(X_train, y_train)\nprint(grid.best_estimator_)\n"

LGBMRegressor(n_estimators=150, num_leaves=35, random_state=42)

In [31]:
%%time

# Обучение модели LGBMRegressor
lgb = LGBMRegressor(n_estimators=150, num_leaves=35, random_state=42)
lgb_pipe = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', lgb)])
lgb_pipe.fit(X_train, y_train)

CPU times: user 7.37 s, sys: 29.4 ms, total: 7.4 s
Wall time: 7.36 s


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric_transformer',
                                                  StandardScaler(),
                                                  ['RegistrationYear', 'Power',
                                                   'Kilometer',
                                                   'RegistrationMonth']),
                                                 ('categorical_transformer',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['VehicleType', 'Gearbox',
                                                   'Model', 'FuelType', 'Brand',
                                                   'Repaired'])])),
                ('regressor',
                 LGBMRegressor(n_estimators=150, num_leaves=35,
                          

In [32]:
%%time

# Предсказания модели LGBMRegressor
y_pred_lgb = lgb_pipe.predict(X_valid)

rmse_lgb = mean_squared_error(y_valid, y_pred_lgb) ** 0.5
print("LGBMRegressor RMSE: {:.3f}".format(rmse_lgb))

LGBMRegressor RMSE: 1644.850
CPU times: user 490 ms, sys: 0 ns, total: 490 ms
Wall time: 501 ms


<a name='analysis'></a>
## 3. Анализ моделей
[Вернуться во Введение](#introduction)

Рассмотрено 3 модели с соответствующими гиперпараметрами, а также значениями метрики RMSE и временем обучения:
- LinearRegression():
    - RMSE: 2889.069;
    - Время обучения: 377 ms;
    - Время предсказания: 125 ms.

- DecisionTreeRegressor(max_depth=16, min_samples_split=20, random_state=42):
    - RMSE: 1841.436;
    - Время обучения: 1.18 s;
    - Время предсказания: 75 ms.

- LGBMRegressor(n_estimators=150, num_leaves=35, random_state=42):
    - RMSE: 1644.850;
    - Время обучения: 8.7 s;
    - Время предсказания: 524 ms.

Наилучшее значение метрики RMSE демонстрирует модель LGBMRegressor. Вдобавок, скорость обучения и предсказания не столь высока, чтобы была сильно заметна пользователю. Ввиду чего, проверим на тестовой выборке именно модель LGBMRegressor.

In [33]:
%%time

# LGBMRegressor
y_pred_final = lgb_pipe.predict(X_test)
rmse = mean_squared_error(y_test, y_pred_final) ** 0.5
print("LGBMRegressor RMSE: {:.3f}".format(rmse))

LGBMRegressor RMSE: 1661.995
CPU times: user 493 ms, sys: 0 ns, total: 493 ms
Wall time: 496 ms


<a name='conclusion'></a>
## 4. Вывод
[Вернуться во Введение](#introduction)

LinearRegression не выполняет необходимые требования касательно допустимого значения метрики RMSE (< 2500). DecisionTreeRegressor хорошо справляется с поставленной задачей, демонстрируя значение RMSE в районе 1844. В дополнение, у нее довольно быстрая скорость обучения и предсказания. Однако, лучше c метрикой RMSE справляется модель LGBMRegressor, значение которой составляет 1644 на валидационной выборке. Также, незначительное повышение в скорости предсказания не столь заметно было бы для пользователя, поэтому лучшим решением было бы использовать именно ее.

***Рекомендация***: 

- **LGBMRegressor(n_estimators=150, num_leaves=35)** - лучшее значение метрики RMSE на валидационной выборке - **1644** (на почти 200 пунктов меньше DecisionTreeRegressor).